## Руководство по решению

Вам необходимо провести описанные ниже два этапа исследования, написать по результатам каждого этапа небольшой отчет (ниже указаны вопросы, ответы на которые должны содержаться в отчете), и предоставить для ревью данный отчет и код, с помощью которого вы выполнили задание.

**Обратите внимание:** высокое качество работы на кросс-валидации (близкое к 100%) — это в первую очередь повод задуматься о том, правильно ли вы обучаете модель. Возможно, вы заглядываете в будущее или настраиваетесь на неправильном наборе признаков.

### Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

1) Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [1]:
import pandas as pd


data = pd.read_csv('features.zip', index_col='match_id')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 114406
Columns: 108 entries, start_time to barracks_status_dire
dtypes: float64(12), int64(96)
memory usage: 80.9 MB


In [3]:
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [4]:
Y = data['radiant_win']

In [5]:
X = data[data.columns[0:-6]]

2) Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [6]:
misses = (Y.count() - X.count())[Y.count() != X.count()]
misses

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

In [7]:
print('\n'.join(misses.index))

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


3) Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [8]:
X.fillna(value=0, inplace=True)

/Users/WLMike/anaconda3/envs/ML36/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


4) Какой столбец содержит целевую переменную? Запишите его название.

In [9]:
print(Y.name)

radiant_win


5) Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [10]:
from sklearn import model_selection
from sklearn import ensemble
import numpy as np
import datetime


for n_estimators in [10, 20, 30, 100]:
    clf = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=31415)
    cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=31415)
    start_time = datetime.datetime.now()
    result = model_selection.cross_validate(
        clf, X, Y,
        cv=cv,
        scoring='roc_auc')
    print('n_estimators =', str(n_estimators))
    print('Time elapsed =', str(datetime.datetime.now() - start_time))
    print('roc_auc =', str(np.mean(result['test_score'])))      
    print('\n')

n_estimators = 10
Time elapsed = 0:00:21.115890
roc_auc = 0.665003066636268


n_estimators = 20
Time elapsed = 0:00:36.221471
roc_auc = 0.6818253362730063


n_estimators = 30
Time elapsed = 0:00:52.382717
roc_auc = 0.6897405323226767


n_estimators = 100
Time elapsed = 0:02:54.505076
roc_auc = 0.706417916525103




In [11]:
clf.set_params(n_estimators=30)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              presort='auto', random_state=31415, subsample=1.0, verbose=0,
              warm_start=False)

In [12]:
clf.fit(X, Y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              presort='auto', random_state=31415, subsample=1.0, verbose=0,
              warm_start=False)

In [13]:
(clf.feature_importances_ != 0).sum() / X.columns.shape[0]

0.21568627450980393


##### Что указать в отчете
В отчете по данному этапу вы должны ответить на следующие вопросы:

1) Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

first_blood_time

first_blood_team

first_blood_player1

first_blood_player2 - первая кровь не произошло за 5 минут матча или второй игрок не был задействован в этом событии

radiant_bottle_time

radiant_courier_time - событие не произошло за 5 минут матча

radiant_flying_courier_time

radiant_first_ward_time

dire_bottle_time

dire_courier_time

dire_flying_courier_time

dire_first_ward_time


2) Как называется столбец, содержащий целевую переменную?

radiant_win

3) Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.

Время кросс-валидации для 30 деревьев около 52c

AUC-ROC = 0.68974

4) Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Так как AUC-ROC продолжант увеличиваться для 10, 20 и 30 деревьев следует использовать больше деревьев. Увеличение числа деревьев до 100 показывает, что рост качества продолжается. 

Для ускорения обучения при большем числе деревьев можно использовать параметры subsample и max_features, что ускорит обучение отдельных деревьев. Кроме того, для построения 30 деревьев используется только 22% признаков - для ускорения вычислений можно исключить признаки с низкой важностью.

### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

**Важно:** не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

In [14]:
from sklearn import preprocessing

X_norm = preprocessing.scale(X)

/Users/WLMike/anaconda3/envs/ML36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


1) Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [15]:
from sklearn import linear_model
import datetime


for C in [10 ** (i/3) for i in range(-9, -2)]:
    clf = linear_model.LogisticRegression(C=C, random_state=31415)
    cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=31415)
    start_time = datetime.datetime.now()
    result = model_selection.cross_validate(
        clf, X_norm, Y,
        cv=cv,
        scoring='roc_auc')
    print('C = ' + str(C))
    print('Time elapsed =', str(datetime.datetime.now() - start_time))
    print('roc_auc = ' + str(np.mean(result['test_score'])))      
    print('\n')

C = 0.001
Time elapsed = 0:00:07.372203
roc_auc = 0.7162908542056704


C = 0.0021544346900318843
Time elapsed = 0:00:08.403757
roc_auc = 0.7164721956469058


C = 0.004641588833612777
Time elapsed = 0:00:09.024873
roc_auc = 0.7165186513089421


C = 0.01
Time elapsed = 0:00:09.594000
roc_auc = 0.7165178837582659


C = 0.021544346900318832
Time elapsed = 0:00:09.198617
roc_auc = 0.7165114362951417


C = 0.046415888336127795
Time elapsed = 0:00:09.447098
roc_auc = 0.7165076433646139


C = 0.1
Time elapsed = 0:00:10.264804
roc_auc = 0.7165039973683055




2) Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [16]:
cat_col = ['lobby_type',
           'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
           'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

X_norm = preprocessing.scale(X[[col for col in X.columns if col not in cat_col]])

for C in [10 ** (i/3) for i in range(-9, -2)]:
    clf = linear_model.LogisticRegression(C=C, random_state=31415)
    cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=31415)
    start_time = datetime.datetime.now()
    result = model_selection.cross_validate(
        clf, X_norm, Y,
        cv=cv,
        scoring='roc_auc')
    print('C = ' + str(C))
    print('Time elapsed =', str(datetime.datetime.now() - start_time))
    print('roc_auc = ' + str(np.mean(result['test_score'])))      
    print('\n')

/Users/WLMike/anaconda3/envs/ML36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


C = 0.001
Time elapsed = 0:00:06.161143
roc_auc = 0.7163053452339199


C = 0.0021544346900318843
Time elapsed = 0:00:06.428104
roc_auc = 0.7164957176054803


C = 0.004641588833612777
Time elapsed = 0:00:07.585397
roc_auc = 0.7165333848230766


C = 0.01
Time elapsed = 0:00:07.905369
roc_auc = 0.7165347300860666


C = 0.021544346900318832
Time elapsed = 0:00:08.459842
roc_auc = 0.7165251371667134


C = 0.046415888336127795
Time elapsed = 0:00:08.550617
roc_auc = 0.716519921093752


C = 0.1
Time elapsed = 0:00:08.775752
roc_auc = 0.7165164587182187




3) На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [17]:
all = set()

for col in cat_col[1:]:
    unique = X[col].unique()
    print(col, unique.shape[0])
    all = all | set(unique)

print('all', len(all))

r1_hero 108
r2_hero 108
r3_hero 108
r4_hero 108
r5_hero 108
d1_hero 108
d2_hero 108
d3_hero 108
d4_hero 108
d5_hero 108
all 108


4) Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [18]:
cat_col = ['lobby_type',
           'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
           'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

scaler = preprocessing.StandardScaler()
X_norm = scaler.fit_transform(X[[col for col in X.columns if col not in cat_col]])

X_pick = np.zeros((data.shape[0], len(all)))
heroes = list(all)

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, heroes.index(data.loc[match_id, 'r%d_hero' % (p+1)])] = 1
        X_pick[i, heroes.index(data.loc[match_id, 'd%d_hero' % (p+1)])] = -1

In [19]:
X_norn_pick = np.hstack((X_norm, X_pick))

5) Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [20]:
for C in [10 ** (i/3) for i in range(-6, 1)]:
    clf = linear_model.LogisticRegression(C=C, random_state=31415)
    cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=31415)
    start_time = datetime.datetime.now()
    result = model_selection.cross_validate(
        clf, X_norn_pick, Y,
        cv=cv,
        scoring='roc_auc')
    print('C = ' + str(C))
    print('Time elapsed =', str(datetime.datetime.now() - start_time))
    print('roc_auc = ' + str(np.mean(result['test_score'])))      
    print('\n')

C = 0.01
Time elapsed = 0:00:13.067101
roc_auc = 0.7515552270368531


C = 0.021544346900318832
Time elapsed = 0:00:15.493033
roc_auc = 0.7517410623301751


C = 0.046415888336127795
Time elapsed = 0:00:17.800194
roc_auc = 0.7517736284262794


C = 0.1
Time elapsed = 0:00:16.920848
roc_auc = 0.7517696054860584


C = 0.2154434690031884
Time elapsed = 0:00:17.502873
roc_auc = 0.7517633092549703


C = 0.4641588833612779
Time elapsed = 0:00:18.194913
roc_auc = 0.7517582521713415


C = 1.0
Time elapsed = 0:00:18.579659
roc_auc = 0.7517551078390803




6) Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [21]:
clf = linear_model.LogisticRegression(C=10 ** (-4 /3), random_state=31415)
clf.fit(X_norn_pick, Y)

LogisticRegression(C=0.046415888336127795, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=31415,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [22]:
test = pd.read_csv('features_test.zip', index_col='match_id')
test.fillna(value=0, inplace=True)
X_test = scaler.transform(test[[col for col in test.columns if col not in cat_col]])

X_test_pick = np.zeros((test.shape[0], len(all)))

for i, match_id in enumerate(test.index):
    for p in range(5):
        X_test_pick[i, heroes.index(test.loc[match_id, 'r%d_hero' % (p+1)])] = 1
        X_test_pick[i, heroes.index(test.loc[match_id, 'd%d_hero' % (p+1)])] = -1

X_test_full = np.hstack((X_test, X_test_pick))

results = clf.predict_proba(X_test_full)

In [23]:
results.min()

0.0036961271801949636

In [24]:
results.max()

0.996303872819805

##### Что указать в отчете
В отчете по данному этапу вы должны ответить на следующие вопросы:

1) Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

ROC_AUC = 0.71652 - качество достаточно слабо зависит от коэффциента регуляризации при изменении его на два порядка от 0.001 до 0.1. Максимум качество достигается вблизи 0.00464.

Качество логистической регрессии выше, чем у градиентного бустинга при 30 деревьях.

Причина отставания градиентного бустинга в его медленной обучаемости - качество продолжает достаточно устойчиво расти по мере увеличения количества деревьев. Вероятно, при достаточном количестве деревьев градиентный бустин даст лучший результат, чем логистическая регрессия. 

Логистическая регрессия относится к линейным методам, как следствие работает заметно быстрее градиентного бустинга (даже для всего 10 деревьев).

2) Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Удаление категориальных признаков повысило в пятом знаке ROC_AUC = 0.71663.


Данные признаки вряд ли возможно разделить линейной разделяющей поверхностью, поэтому они являются шумовыми для логистической регрессии. Их удаление ведет к снижению рисков переобучения и к незначительному улучшению качества. 

3) Сколько различных идентификаторов героев существует в данной игре?

В игре существует 108 героев

4) Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Качество увеличилось ROC_AUC=0.75177.

Можно предположить, что герои не равноценны - мешок слов позволяет оценить ценность отдельных героев и повысить качество классификации.

5) Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

Минимальная вероятность победы 0.0037

Максимальная вероятность победы 0.9963